### Import Dependecies

In [1]:
import numpy as np
import cv2
from math import ceil
import sounddevice as sd
from time import sleep

### Pre-process Image

In [2]:
def input_image(file_path):
    img=cv2.imread(file_path)
    return cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

#### convert image to 256x256 if it is larger

In [3]:
def resize(image):
    #if img size is greater than 256x256 we should resize
    #print(image.shape)
    width,height=image.shape[0],image.shape[1]
    if width>256 and height<256:
        new_image=cv2.resize(image,(256,height))
    elif height>256 and width<256:
        new_image=cv2.resize(image,(width,256))
    elif width>256 and height>256:
        new_image=cv2.resize(image,(256,256))
    
    new_image=new_image.astype(np.float32)
    new_image=new_image.round(3)
    
    return new_image

## Convert Image to Sound

### map Hue ranges to musical note frequencies

In [4]:
hue2notes={
    0:0,
    1: 16.35,      #"C0"
    2: 17.32,      #"C#0"
    3: 18.35,      #"D0"
    4: 19.45,      #"E_flat0
    5: 20.6,      #"E0"
    6: 21.83,      #"F0"
    7:23.12,      #"F#0"
    8: 24.5,      #"G0"
    9: 25.96,      #"A_flat0
    10: 27.5,      #"A0"
    11:29.14,      #"B_flat0
    12:30.87,      #"B0"
}

### Convert Hue to Note frequency

In [5]:
def hue2note(hue):
    hue_percent=hue/179
    note=hue_percent*12
    note_index=np.round(note)
    #print(note_index)
    freq=hue2notes[note_index]
    
    return freq


### Convert Saturation to Octave value

In [6]:
def s2note(saturation):
    if saturation==0: return 0
    else:
        octave=saturation/32

    return ceil(octave)


### Convert Value (or lightness) to Amplitude

In [7]:
def v2note(value):
    if value==0: return 0
    else:
        amplitude=value/255
        amplitude="{:.2f}".format(amplitude)

    return amplitude


### Modify Hue values of Array to Frequencies

In [8]:
def modify_array_hue(array):
  for i in range(array.shape[0]):
    for j in range(array.shape[1]):  
      array[i,j,0] = hue2note(array[i,j,0])

  return array

### Modify Saturation values of Array to Octave

In [9]:
def modify_array_saturation(array):
  for i in range(array.shape[0]):
    for j in range(array.shape[1]):  
      array[i,j,1] = s2note(array[i,j,1])

  return array

### Modify Values (or lightness) of Array to Octave

In [10]:
def modify_array_value(array):
  for i in range(array.shape[0]):
    for j in range(array.shape[1]):  
      array[i,j,2] = v2note(array[i,j,2])

  return array

## Pre-process Sound

### Convert Pixel to frequency

In [11]:
def px2freq(array):
    freq=array[0]*(array[1]*(2**array[1]))
    return freq


### Create Co-sinusodial wave

In [16]:
def make_wave(freq, duration,amplitude,sample_rate = 44100,):
    wave = []

    for i in range(0,int(duration*sample_rate)):
        wave.append(i/((sample_rate/(2*np.pi))/freq))

    wave = amplitude*np.cos(np.stack(wave))
    return wave

### Create Chord with two notes (two pixels)

In [22]:
def chord(array):
  a=[]
  for col in range(0,array.shape[1]):
    for row in range(array.shape[0]):  
      freq=px2freq(array[row,col])
      a.append(freq)
    chord0=make_wave(a[0],0.5,array[row,col,0])
    chord1=make_wave(a[1],0.5,array[row,col,1])
    chord_sum=chord0+chord1
    chord_sum = chord_sum*0.1/np.max(chord_sum) #to prevent the volume (amplitude) to be beyond 1 # to prevent clipping of sound
    sd.play(chord_sum,22050)
    a.clear()
    sleep(.9)
  
  return "Done"

## Example
##### Create a deep copy before you commence operation to avoid overwriting image file

In [23]:
img=input_image("img/img_2.png")
new_img=resize(img)
deep_img_copy=new_img.copy()
modify_array_hue(deep_img_copy)
modify_array_saturation(deep_img_copy)
modify_array_value(deep_img_copy)
chord(deep_img_copy)

C:\Users\Iminabo\AppData\Local\Temp\ipykernel_19516\670263659.py:5: RuntimeWarning: divide by zero encountered in scalar divide
  wave.append(i/((sample_rate/(2*np.pi))/freq))
